In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)

        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        #Max pooling over a (2,2) windows
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        #If the size is a square, you can specify with single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = torch.flatten(x,1) #flatten all dimension except the batch dimention

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) 

10
torch.Size([6, 1, 5, 5])


In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)


tensor([[ 0.0147,  0.0260, -0.0691, -0.1008, -0.0090,  0.0032, -0.0577, -0.0230,
         -0.0988,  0.0813]], grad_fn=<AddmmBackward0>)


In [8]:
net.zero_grad()
out.backward(torch.randn(1, 10))

Adding Loss funtions

In [9]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7111, grad_fn=<MseLossBackward0>)


In [11]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [12]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
